In [1]:
import keras
import numpy as np

path = keras.utils.get_file("nietzsche.txt", origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read().lower()

print("Corpus length:", len(text))

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

606208/600901 [==============================] - 2s 4us/step
Corpus length: 600893


In [2]:
maxlen = 60
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print("Number of sequences:", len(sentences))

chars = sorted(list(set(text)))
print("Unique characters:", len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print("Vectorization...")
# one-hot encodes
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


In [3]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation="softmax"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               95232     
_________________________________________________________________
dense_1 (Dense)              (None, 57)                7353      
Total params: 102,585
Trainable params: 102,585
Non-trainable params: 0
_________________________________________________________________


In [4]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [5]:
# sampling function
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # 通过多项式随机选取预测结果。
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [11]:
import random
import sys

for epoch in range(1, 60):
    print("epoch", epoch)
    model.fit(x, y, batch_size=128, epochs=1)
    # 从text中随机抽出一个片段作为seed。
    start_index = random.randint(0, len(text) - maxlen -1)
    generated_text = text[start_index: start_index + maxlen]
    print("------Generating with seed: "" + generated_text + """ )
    # temperature越大，选择的自由度越高，结果越难以预测。反之，结果易出现重复。
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print("------ temperature:", temperature)
        # print等同于sys.stdout.write(obj+'\n')，此处不希望换行符。
        sys.stdout.write(generated_text)
        
        # 在seed的基础上生成400字母。
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                # one-hot encode
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)

epoch 1

Epoch 1/1
200278/200278 [==============================] - 139s 694us/step - loss: 1.9550
------Generating with seed:  + generated_text + 
------ temperature: 0.2
--is no longer either desired or fully developed, is
reasonable and the conscience and and and who would the conscience and in the soul the conscience of the another who propers and the soul the most soul the conscience of the arthess to the ancient the decertain the moral the art the world to the conscience of the conscience and soul the ancation of the all the soul the world the most consequent to the action of the decertain the conscience of the another------ temperature: 0.5
to the action of the decertain the conscience of the another and in all the proceess in the was as wortherly methoring and concernal soul the ormin this the fance, who he some how and the conscience of the most this constither to the concead unconsequently in more the constither to restrom a soul the pastered to the faine to the for this decr

the complietians, philosophclaure of self that be hand--that a unifr itternnisamaing moast, time for thus rangwarius vuratition of rcarts whichselatition, something into deally. their muscraes or time:
long; the essential mofe obciedity that swilk-a1d entys toog-culitate of the crhusiwn to compra8; it are by laquide ardenge). roctidesx of shord and grisari, as a prdy fullty incivilation, by the
satisual conception, their apuling,
of nampoicued, about was aepoch 5
Epoch 1/1
200278/200278 [==============================] - 129s 645us/step - loss: 1.4472
------Generating with seed:  + generated_text + 
------ temperature: 0.2
resent-day european,
will always elicit the same imperative of the soul is not and the strength and the discoveres of the conscience of the strength of the spirit the single of the singuistic and the strength of the strength of the strength of the same the strength of the compare of the disconditions of the spirit and in the strength of the senses of the strength of 

somethical"" of the love and orgolicable thats believe of a
hondoush time it can religious cannots and lights good that the most
para------ temperature: 1.2
 it can religious cannots and lights good that the most
parable fvoulecyly another--and everything var hortial and errisontion of the
unhavealt to posse. i he mustwompals libercurmy shateftinateness of every uneledis of kind therefor worsi, must, it is", we is unour licalo) every will must furbed compal as so
differert
for a roadke. now best over scquetxent, hali1ocant again a exomptions of
the precouse his other dues occasitually ba
firred and
disticular epoch 9
Epoch 1/1
200278/200278 [==============================] - 129s 642us/step - loss: 1.3843
------Generating with seed:  + generated_text + 
------ temperature: 0.2
ies prevail, chiefly because they must prevail, or else
run man and all the same the more and all the same and the stronger of the expression of the stronger of the strange of the sentiment and the desire to the 

rook for other more lowness, and in "ideas in chars and honesty, influence--this trance only be lavely. but
child blee as a being when all a pleofmer. the soulsbocity of se,
may be power existence in persis soble of "pictuse? also, with a "en, is factbation to all bad and isterists stome pa------ temperature: 1.2
 with a "en, is factbation to all bad and isterists stome pay,
is symptom to er natings,--ip causely and 
lought causent, through the expression embratiaritying whereverthes inkepletellarted frol have dor from haptible pursee of all his feate? in self"--for sliked anotherom
fonly suffering, our.w
a"scuperoim her order olname
to stren" its graps
own boo""?"!--all move longous buraucation if they artalt had something way of made to soul. as probabon listuregepoch 13
Epoch 1/1
200278/200278 [==============================] - 129s 644us/step - loss: 1.3526
------Generating with seed:  + generated_text + 
------ temperature: 0.2
ive (perhaps disturbing and destroying)
experiences, 

 a great presenters of the contenter belief is a more spirit of mere the dangerous when his contentery ble" of premate with repued but also throushed, as thrult this be
forehure
of wicher still comprehimnction of
hormant and "free gdea rary all, i glarting-mistor that they dreadgener mestive cannot line stranges to its owerny as is refering for their much for their
me, one one, wherever circumning as moral earce this voluncation music, of it are under a ta------ temperature: 1.2
 as moral earce this voluncation music, of it are under a taste is demands
the loses und formjwor, agreeably the truth from them terres mly all thirsenantical yemant afs, he genture"s they. madex for there,
 men ney, ervial
mosts braipted with the men a dable causage be
 os confings, the view
of exacversulled havinard, aptanciously as let a best has anore themselves but blamal, "someremay
demand tece ariss trus coleborable--from had amorvaliwile, party. ofepoch 17
Epoch 1/1
200278/200278 [======================

...

ct and the sense of the contempt the man and interest and the contempt the conduct: the developed that it were complication of the intellectual discipliarity and far as a souls and like, and the infertion, even that the science with the sentiment with the states of the same all so high a finer the higher interests and state of heregors and sacrifice of the reason in the acts a man, and an anticatical consequently in the existence of the sentiment in the or------ temperature: 1.0
cal consequently in the existence of the sentiment in the ordinary is of the life.=--one bequisher
often conditionality, an invoiviture himself.

ingien of order of the experience by the austh fafulative correatwically tafted in much and
epicurent what no longer--or convenser threatacha, and if the philosope evils; cononce his in-is is eventness sripticsly now even and long ip yet that disattation before liesd the want of modern imporareness, who habit of ------ temperature: 1.2
 before liesd the want of modern

humanity (not alone as an individual) frit free of the same and like the strength of the same the sense of the strength of the superficial conditions of the sense of the standard and from our vanity of the strength and such a sounds of the manifests of the world also and all the sense and forergnes of the same the strength of the mankind and also and all the spirit and something which has hitherto been the states of the sense of the same------ temperature: 0.5
 which has hitherto been the states of the sense of the same my long belongere and propering and make them from a great proper and extravently and claim and only the spiritual suffering and also and possible or a soul of the conditions of the most endured and steral condition of the powerful of the persvetued at all and the most changes and has once will be religion of the self-destruct of the case of respect to men who has a sension of state of the always------ temperature: 1.0
e of respect to men who has a sension of state of t

200278/200278 [==============================] - 126s 629us/step - loss: 1.2390
------Generating with seed:  + generated_text + 
------ temperature: 0.2
ar
displays of resentment have been made against pictures of the strength of the strength of the same and the strength of the most conscience of the same and of the same things are the most conscience of the sense of the sense of the same one warn the sense of the same things are belief and the fact the destruction and the most personal self-soul of the spirit of the sentiments of the state of the states of the sense of the spirit of the same and the state------ temperature: 0.5
 states of the sense of the spirit of the same and the states of christian to consider the hardless of our higher that the great even seems to everything the states when they look or such an our conscience of the suffers to discontent the last the sense of the senses of the reason of this case of the delight and all the most death of the father as intellect man

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


y to creature--which is satisfacti